# Run Theta Model
Steps:
1. Open dataset file
2. Run Theta model (including save to file)
3. Evaluation

# Step 1. Read dataset file

## Import libraries

In [1]:
# Import file and data libraries
import math
import pickle
import numpy as np
import pandas as pd 
import collections
import matplotlib.pyplot as plt

# Import exponential smoothing model
from statsmodels.tsa.holtwinters import ExponentialSmoothing 

# Import Matthews Correlation Coefficient module (MCC)
from sklearn.metrics import matthews_corrcoef

import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Read dataset file

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Read X & Y datasets from Pickle file, along with the metadata
dataset_path = "/content/drive/My Drive/Colab Notebooks/3. Code/3. Compare/Dataset/"

with open(dataset_path + "Dataset V2.pickle", "rb") as f:
    X = pickle.load(f)
    Y = pickle.load(f)  # Contains Y_Avg, X_Last, Y_Diff, Y_Band, Y_Gap, Y_Days
    Metadata = pickle.load(f) # Contains the Company ID, ticker, sector and Dataset ID
f.close()

In [4]:
X[0]

Date    HL_Avg     Volume  HL_Avg_Norm  Volume_Norm
1824 1988-03-02  0.395089  292320000     0.001910     0.039388
1825 1988-03-03  0.408482  473760000     0.001984     0.063835
1826 1988-03-04  0.412946  209440000     0.002009     0.028220
1827 1988-03-07  0.420759  207200000     0.002052     0.027918
1828 1988-03-08  0.415179  144480000     0.002021     0.019467
...         ...       ...        ...          ...          ...
3083 1993-02-23  0.487723  194073600     0.002422     0.026150
3084 1993-02-24  0.473214  286563200     0.002342     0.038612
3085 1993-02-25  0.482143  167227200     0.002391     0.022532
3086 1993-02-26  0.475446  294884800     0.002354     0.039733
3087 1993-03-01  0.474330  119302400     0.002348     0.016075

[1264 rows x 5 columns]

# Step 2. Create Theta lines

In [5]:
# Set the coefficients for each Theta line
Theta1_Coef = 0.25  # The coefficient for Theta1 line
Theta2_Coef = 0.5   # The coefficient for Theta2 line
Theta3_Coef = 0.75  # The coefficient for Theta3 line
Theta4_Coef = 1.25  # The coefficient for Theta4 line
Theta5_Coef = 1.5   # The coefficient for Theta5 line
Theta6_Coef = 2     # The coefficient for Theta6 line

In [6]:
# Function to calculate a Theta line 
def Calculate_Theta_Line (Theta_Coef, Theta_Column):
    Theta_Diff2 = Diff2_Today * Theta_Coef
    Theta_Diff1 = Theta_Diff2 + Diff1_Today
    Theta_Price = Theta_Diff1 + Price_Prev
    X[DS][Theta_Column].iloc[day-2] = Theta_Price
    return 0

In [7]:
# Define the number of samples once
Number_of_Samples = 10 #1000

In [8]:
# Calculate Theta Lines for 1000 datasets with 6 Theta lines
# Run time: 35 mins for 1000 datasets (for 6 Theta lines)
for DS in range(Number_of_Samples): 

    # Add columns to calculate Theta lines
    #X[DS]["Diff1"] = np.nan
    #X[DS]["Diff2"] = np.nan
    X[DS]["Theta1"] = np.nan
    X[DS]["Theta2"] = np.nan
    X[DS]["Theta3"] = np.nan
    X[DS]["Theta4"] = np.nan
    X[DS]["Theta5"] = np.nan
    X[DS]["Theta6"] = np.nan

    # Loop for all days in the dataset
    for day in range(1, len(X[DS])): 

        # Calculate the first level difference (starting from day 1, not day 0)
        Price_Today = X[DS]["HL_Avg_Norm"].iloc[day]
        Price_Prev  = X[DS]["HL_Avg_Norm"].iloc[day-1]
        Diff1_Today = Price_Prev - Price_Today
        #X[DS]["Diff1"].iloc[day] = Diff1_Today

        # Calculate the second level difference (starting from day 2, not day 1)
        if day > 1:
            Price_Prev2 = X[DS]["HL_Avg_Norm"].iloc[day-2]
            Diff1_Prev  = Price_Prev2 - Price_Prev
            Diff2_Today = Diff1_Prev - Diff1_Today
            #X[DS]["Diff2"].iloc[day] = Diff2_Today

            # Calculate Theta Lines with different Theta coefficients
            Calculate_Theta_Line(Theta1_Coef, "Theta1")
            Calculate_Theta_Line(Theta2_Coef, "Theta2")
            Calculate_Theta_Line(Theta3_Coef, "Theta3")
            Calculate_Theta_Line(Theta4_Coef, "Theta4")
            Calculate_Theta_Line(Theta5_Coef, "Theta5")
            Calculate_Theta_Line(Theta6_Coef, "Theta6")

    # Monitor the run of 1000 dataset
    print("Dataset:", DS)

Dataset: 0
Dataset: 1
Dataset: 2
Dataset: 3
Dataset: 4
Dataset: 5
Dataset: 6
Dataset: 7
Dataset: 8
Dataset: 9


# Step 3. Store Theta Lines

In [9]:
# Be aware that the last 2 rows the Theta columns are NaN 
# because we use differentiation level 2 (and integration)
X[5]
# Also, only X[0] to X[999] will have Theta lines, X[1000] to X[3988] won't
# The LSTM will be trained on X[1000] to X[3988], so the LSTM won't see the test dataset (0-999)

Date    HL_Avg     Volume  HL_Avg_Norm  Volume_Norm    Theta1  \
2286 1989-12-28  0.310268  151256000     0.001441     0.020380  0.001409   
2287 1989-12-29  0.313058  152409600     0.001457     0.020536  0.001445   
2288 1990-01-02  0.323661  183198400     0.001515     0.024684  0.001562   
2289 1990-01-03  0.337054  207995200     0.001589     0.028026  0.001613   
2290 1990-01-04  0.339286  221513600     0.001602     0.029847  0.001593   
...         ...       ...        ...          ...          ...       ...   
3545 1994-12-20  0.346540  175145600     0.001642     0.023599  0.001591   
3546 1994-12-21  0.339286  157438400     0.001602     0.021213  0.001609   
3547 1994-12-22  0.344308  133078400     0.001630     0.017931  0.001630   
3548 1994-12-23  0.347657   93889600     0.001648     0.012651       NaN   
3549 1994-12-27  0.351004   81916800     0.001667     0.011038       NaN   

        Theta2    Theta3    Theta4    Theta5    Theta6  
2286  0.001420  0.001431  0.001452  0.001463  0.001485  
2287  0.001449  0.001453  0.001461  0.001465  0.001472  
2288  0.001546  0.001531  0.001500  0.001485  0.001454  
2289  0.001605  0.001597  0.001582  0.001574  0.001559  
2290  0.001596  0.001599  0.001605  0.001608  0.001614  
...        ...       ...       ...       ...       ...  
3545  0.001608  0.001625  0.001659  0.001676  0.001710  
3546  0.001606  0.001604  0.001600  0.001597  0.001593  
3547  0.001630  0.001630  0.001630  0.001630  0.001630  
3548       NaN       NaN       NaN       NaN       NaN  
3549       NaN       NaN       NaN       NaN       NaN  

[1264 rows x 11 columns]

In [10]:
# Save X & Y datasets with Theta lines into a Pickle file
dataset_path = "/content/drive/My Drive/Colab Notebooks/3. Code/3. Compare/Dataset/"

with open(dataset_path + "Dataset Theta.pickle", "wb") as f:
    pickle.dump(X, f) #Date, HL_Avg, Volume, HL_Avg_Norm, Volume_Norm, Theta1, Theta2, Theta3, Theta4, Theta5, Theta6
    pickle.dump(Y, f) #Y_Avg, X_Last, Y_Diff, Y_Band, Y_Gap, Y_Days, Y_Avg_Norm, X_Last_Norm, Y_Band_Norm
    pickle.dump(Metadata, f) # Company ID, ticker, sector, Dataset ID
f.close()